# Проект: Классификация новостных статей по тематикам (NLP)

**Автор:** Андрамонов Арсений
**Цель:** Разработать модель машинного обучения для автоматического определения темы новостной статьи на основе ее текста. Проект демонстрирует навыки работы с неструктурированными текстовыми данными (NLP).

**План работы:**
1.  **Загрузка данных:** Используем встроенный в `scikit-learn` датасет **20 Newsgroups**, содержащий ~18,000 статей.
2.  **Предобработка текста (Text Preprocessing):**
    *   Приведение к нижнему регистру, удаление знаков препинания и цифр.
    *   Токенизация (разделение текста на слова).
    *   Удаление неинформативных "стоп-слов".
    *   Лемматизация (приведение слов к их словарной форме).
3.  **Векторизация текста:**
    *   Преобразование очищенных текстов в числовые векторы с помощью метода **TF-IDF**.
4.  **Построение и оценка модели:**
    *   Разделение данных на обучающую и тестовую выборки.
    *   Обучение модели **Логистическая регрессия**.
    *   Оценка качества модели с помощью метрики **Accuracy** и детального отчета по классам.
5.  **Заключение:** Формулирование итоговых выводов по проекту.

## Шаг 1: Загрузка данных из scikit-learn

Для этого проекта мы будем использовать классический NLP-датасет "20 Newsgroups". Его преимущество в том, что он встроен напрямую в библиотеку `scikit-learn`, что позволяет загрузить его одной командой, без необходимости скачивать CSV-файлы.

Датасет состоит из ~18,000 новостных статей, разделенных на 20 различных тематических групп.

In [3]:
# Импортируем базовые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# ИМПОРТИРУЕМ ФУНКЦИЮ ДЛЯ ЗАГРУЗКИ ДАТАСЕТА
from sklearn.datasets import fetch_20newsgroups

print('Библиотеки успешно импортированы!')

# 1. ЗАГРУЖАЕМ ДАННЫЕ
# Scikit-learn сама скачает и закэширует данные при первом запуске
# Мы убираем служебную информацию (headers, footers, quotes), чтобы модель училась только на тексте статей
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

print("\nДанные успешно загружены!")

# 2. СОЗДАЕМ PANDAS DATAFRAME ДЛЯ УДОБСТВА
# Данные загружаются в специальном формате scikit-learn, мы преобразуем их в привычный DataFrame
df = pd.DataFrame({
    'review': newsgroups.data,  # Текст статьи
    'target_id': newsgroups.target # ID категории (число от 0 до 19)
})

# Добавляем названия категорий для наглядности
df['target_name'] = df['target_id'].apply(lambda x: newsgroups.target_names[x])

print(f"Размер датасета: {df.shape[0]} строк и {df.shape[1]} столбцов.")

# 3. УДАЛЯЕМ ПУСТЫЕ СТРОКИ, если они есть
df.dropna(subset=['review'], inplace=True)
df = df[df['review'].str.strip() != '']

# 4. ПОСМОТРИМ НА РЕЗУЛЬТАТ
print("\nРаспределение статей по категориям (первые 10):")
print(df['target_name'].value_counts().head(10))

print("\nПример данных, готовых для обработки:")
# Выведем пример статьи из категории "Космос" (sci.space)
display(df[df['target_name'] == 'sci.space'].head())

Библиотеки успешно импортированы!

Данные успешно загружены!
Размер датасета: 18846 строк и 3 столбцов.

Распределение статей по категориям (первые 10):
target_name
comp.windows.x              982
rec.sport.hockey            975
soc.religion.christian      975
rec.motorcycles             969
comp.sys.ibm.pc.hardware    964
sci.crypt                   962
sci.med                     960
misc.forsale                959
rec.sport.baseball          958
sci.electronics             958
Name: count, dtype: int64

Пример данных, готовых для обработки:


,review,target_id,target_name
25,AW&ST had a brief blurb on a Manned Lunar Exp...,14,sci.space
73,\nThis question comes up frequently enough tha...,14,sci.space
75,\n\n\n\n\n\nPhil> Didn't one of the early jet ...,14,sci.space
112,"""Space Station Redesign Leader Says Cost Goal ...",14,sci.space
158,Original to: keithley@apple.com\nG'day keithle...,14,sci.space


In [4]:
import nltk
import re # Библиотека для работы с регулярными выражениями

# Загружаем необходимые пакеты NLTK
# wordnet - база данных для лемматизации
# stopwords - список стоп-слов
# punkt - токенизатор, который делит текст на предложения/слова
try:
    nltk.data.find('corpora/wordnet.zip')
    nltk.data.find('corpora/stopwords.zip')
    nltk.data.find('tokenizers/punkt.zip')
    print("Необходимые пакеты NLTK уже загружены.")
except nltk.downloader.DownloadError:
    print("Загружаем необходимые пакеты NLTK...")
    nltk.download('wordnet')
    nltk.download('stopwords')
    nltk.download('punkt')
    print("Пакеты успешно загружены.")

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

# Создаем список английских стоп-слов
stop_words = set(stopwords.words('english'))
# Инициализируем лемматизатор
lemmatizer = WordNetLemmatizer()

Необходимые пакеты NLTK уже загружены.


## Шаг 2: Предобработка текста

Это самый важный этап в любом NLP-проекте. Чтобы модель могла эффективно работать с текстом, его необходимо "очистить" от шума и стандартизировать.

Наш пайплайн очистки будет состоять из следующих шагов:
1.  **Нормализация:** Приведение текста к нижнему регистру и удаление всех символов, кроме букв.
2.  **Токенизация:** Разделение сплошного текста на список отдельных слов (токенов).
3.  **Удаление стоп-слов:** Исключение часто встречающихся, но не несущих смысловой нагрузки слов (например, 'a', 'the', 'in').
4.  **Лемматизация:** Приведение каждого слова к его начальной, словарной форме (например, 'running', 'ran' -> 'run'). Это помогает модели понять, что разные формы слова несут один и тот же смысл.

In [5]:
def preprocess_text(text):
    """
    Функция для полной предобработки текста.
    """
    # 1. Приведение к нижнему регистру и удаление всего, кроме букв
    text = re.sub(r'[^a-zA-Z\s]', '', text, re.I|re.A).lower()
    
    # 2. Токенизация (разбиение на слова)
    tokens = word_tokenize(text)
    
    # 3. Лемматизация и удаление стоп-слов
    lemmatized_tokens = [
        lemmatizer.lemmatize(token) for token in tokens if token not in stop_words
    ]
    
    # 4. Соединяем токены обратно в одну строку
    return " ".join(lemmatized_tokens)

# Проверим, как работает наша функция на одном примере
sample_text = df['review'].iloc[0]
cleaned_text = preprocess_text(sample_text)

print("--- Пример работы функции ---")
print("\nОРИГИНАЛЬНЫЙ ТЕКСТ:")
print(sample_text)
print("\nОЧИЩЕННЫЙ ТЕКСТ:")
print(cleaned_text)

--- Пример работы функции ---

ОРИГИНАЛЬНЫЙ ТЕКСТ:


I am sure some bashers of Pens fans are pretty confused about the lack
of any kind of posts about the recent Pens massacre of the Devils. Actually,
I am  bit puzzled too and a bit relieved. However, I am going to put an end
to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they
are killing those Devils worse than I thought. Jagr just showed you why
he is much better than his regular season stats. He is also a lot
fo fun to watch in the playoffs. Bowman should let JAgr have a lot of
fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final
regular season game.          PENS RULE!!!



ОЧИЩЕННЫЙ ТЕКСТ:
sure bashers pen fan pretty confused lack kind post recent pen massacre devil actually bit puzzled bit relieved however going put end nonpittsburghers relief bit praise pen man killing devil worse thought jagr showed m

In [6]:

from tqdm.auto import tqdm
# Регистрируем использование tqdm для pandas
tqdm.pandas()

print("Начинаем предобработку всего датасета. Это может занять несколько минут...")

# Создаем новую колонку с очищенным текстом
# .progress_apply() работает как .apply(), но показывает progress bar
df['review_cleaned'] = df['review'].progress_apply(preprocess_text)

print("\nПредобработка завершена!")
display(df[['review', 'review_cleaned', 'target_name']].head())

Начинаем предобработку всего датасета. Это может занять несколько минут...


  0%|          | 0/18331 [00:00<?, ?it/s]


Предобработка завершена!


,review,review_cleaned,target_name
0,\n\nI am sure some bashers of Pens fans are pr...,sure bashers pen fan pretty confused lack kind...,rec.sport.hockey
1,My brother is in the market for a high-perform...,brother market highperformance video card supp...,comp.sys.ibm.pc.hardware
2,\n\n\n\n\tFinally you said what you dream abou...,finally said dream mediterranean new area grea...,talk.politics.mideast
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,think scsi card dma transfer disk scsi card dm...,comp.sys.ibm.pc.hardware
4,1) I have an old Jasmine drive which I cann...,old jasmine drive use new system understanding...,comp.sys.mac.hardware


## Шаг 3: Векторизация и построение модели

После очистки тексты готовы к преобразованию в числовой формат, понятный для алгоритмов машинного обучения.

### 3.1. Векторизация с помощью TF-IDF

Мы используем метод **TF-IDF (Term Frequency-Inverse Document Frequency)**. Он представляет каждый текст в виде вектора, где вес каждого слова зависит не только от его частоты в данном тексте, но и от его редкости во всем корпусе текстов. Это позволяет выделить ключевые, тематические слова и снизить вес общеупотребительных слов.

### 3.2. Обучение и оценка модели

В качестве классификатора мы будем использовать **Логистическую регрессию**, так как она является мощной и быстрой базовой моделью для текстовых задач. Качество модели будем оценивать по метрике **Accuracy** (доля правильных ответов) на тестовой выборке.

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# 1. Определяем признаки (X) и целевую переменную (y)
# Используем очищенный текст и числовые ID категорий
X = df['review_cleaned']
y = df['target_id']

# 2. Разделяем данные на обучающую и тестовую выборки (80/20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    stratify=y # Важно для сохранения баланса классов
)

print(f"Размер обучающей выборки: {len(X_train)}")
print(f"Размер тестовой выборки: {len(X_test)}")
print("-" * 30)


# 3. Векторизация текста
# Создаем TF-IDF векторизатор
# max_features=5000 говорит, чтобы он использовал только 5000 самых популярных слов. 
# Это ускоряет обучение и часто улучшает качество, убирая очень редкие слова.
vectorizer = TfidfVectorizer(max_features=5000)

# Обучаем векторизатор на тренировочных данных и преобразуем их в векторы
X_train_vec = vectorizer.fit_transform(X_train)

# Преобразуем тестовые данные, используя уже обученный векторизатор
X_test_vec = vectorizer.transform(X_test)

print(f"Тексты преобразованы в матрицу размером: {X_train_vec.shape}")
print("-" * 30)


# 4. Обучение модели
print("Начинаем обучение модели (Логистическая регрессия)...")
# C=10 - параметр регуляризации, подобранный как хороший для этой задачи
# max_iter=1000 - увеличиваем количество итераций для сходимости
model = LogisticRegression(C=10, max_iter=1000, random_state=42)

# Обучаем модель на векторизованных данных
model.fit(X_train_vec, y_train)
print("Обучение завершено!")
print("-" * 30)


# 5. Оценка качества модели
# Делаем предсказания на тестовой выборке
y_pred = model.predict(X_test_vec)

# Считаем точность (Accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"Итоговая точность (Accuracy) на тестовой выборке: {accuracy:.4f}")

# Выводим детальный отчет по всем классам
# target_names=newsgroups.target_names позволяет видеть названия тем, а не просто цифры
print("\nДетальный отчет о классификации:")
print(classification_report(y_test, y_pred, target_names=newsgroups.target_names))

Размер обучающей выборки: 14664
Размер тестовой выборки: 3667
------------------------------
Тексты преобразованы в матрицу размером: (14664, 5000)
------------------------------
Начинаем обучение модели (Логистическая регрессия)...
Обучение завершено!
------------------------------
Итоговая точность (Accuracy) на тестовой выборке: 0.6970

Детальный отчет о классификации:
                          precision    recall  f1-score   support

             alt.atheism       0.54      0.55      0.55       156
           comp.graphics       0.65      0.70      0.68       191
 comp.os.ms-windows.misc       0.62      0.64      0.63       189
comp.sys.ibm.pc.hardware       0.59      0.58      0.58       193
   comp.sys.mac.hardware       0.68      0.69      0.68       186
          comp.windows.x       0.80      0.78      0.79       197
            misc.forsale       0.80      0.66      0.72       192
               rec.autos       0.68      0.68      0.68       187
         rec.motorcycles      

## Шаг 4: Заключение и выводы

В ходе проекта была успешно разработана модель для многоклассовой классификации текстов.

**Ключевые результаты:**
*   Итоговая модель на основе Логистической регрессии и TF-IDF достигла точности **(Accuracy) ~70%** на тестовой выборке при классификации на 20 различных классов. Учитывая, что случайное угадывание дало бы всего 5%, полученный результат является очень хорошим.
*   Анализ детального отчета показал, что модель наиболее успешно классифицирует темы с уникальной и специфичной лексикой (например, `rec.sport.hockey`, `sci.crypt`).
*   Наибольшие трудности возникают при разделении близких по смыслу и лексическому составу тем (например, разные категории компьютерного "железа" или темы, связанные с политикой и религией).

Проект демонстрирует владение полным циклом NLP-задачи: от предобработки сырого текста до построения и интерпретации результатов работы модели-классификатора.